# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key as gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../output_data/cities.csv')
cities.head()

,Unnamed: 0,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,turukhansk,100.0,1605426368,89,65.82,87.98,-12.91,9.71
1,1,hermanus,88.0,1605426212,62,-34.42,19.23,66.00,4.61
2,2,barrow,90.0,1605426368,73,71.29,-156.79,21.20,14.99
3,4,georgetown,20.0,1605426369,74,5.41,100.34,87.80,9.17
4,5,chizhou,0.0,1605426369,46,30.66,117.48,71.04,4.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
coordinates = cities.loc[:,['Lat', 'Lng']]
coordinates = coordinates.fillna(0)
humidity = cities.loc[:,['Humidity']]
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity['Humidity'], dissipating=False, max_intensity= 10, point_radius=0.5)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Eliminate non ideal weather conditions
prio_df = pd.DataFrame(cities, columns = ['City', 'Cloudiness', 'Max Temp', 'Wind Speed', 'Lat', 'Lng', 'Country'])
#prio_df = prio_df.dropna()
ideal_wind = (prio_df['Wind Speed'] < 6)
ideal_cloud = (prio_df['Cloudiness'] == 0)
ideal_temp = (prio_df['Max Temp'] <=80) & (prio_df['Max Temp'] >= 70)

ideal_df = prio_df[ideal_cloud & ideal_temp & ideal_wind]

ideal_df.head()

,City,Cloudiness,Max Temp,Wind Speed,Lat,Lng,Country


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(cities, columns = ['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,turukhansk,NaN,65.82,87.98,
1,hermanus,NaN,-34.42,19.23,
2,barrow,NaN,71.29,-156.79,
3,georgetown,NaN,5.41,100.34,
4,chizhou,NaN,30.66,117.48,
...,...,...,...,...,...
537,mana,NaN,5.66,-53.78,
538,chenzhou,NaN,25.80,113.03,
539,kalmunai,NaN,7.42,81.82,
540,bambanglipuro,NaN,-7.95,110.28,


In [15]:
hotel = []
params = {"radius": 5000, "types": "hotel", "keyword": "hotel", "key": gkey}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params['location']= f'{lat},{lng}'
    map_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    address = requests.get(map_url, params=params)
    address = address.json()
    print(address)
    try:
        hotel.append(address['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 65.80129769999999, 'lng': 87.9712102}, 'viewport': {'northeast': {'lat': 65.80269492989271, 'lng': 87.97341985}, 'southwest': {'lat': 65.79999527010726, 'lng': 87.97047365000002}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'name': 'ООО "Туруханская пушнина"', 'opening_hours': {'open_now': False}, 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109211917074966941380">Александр Ожигов</a>'], 'photo_reference': 'ATtYBwLPc8Mn_UqXHgppCZb0_XaX6vrCeBY-ofs-fVjgIG_PwgJF1EMEulOuOBybdhh674LZIdUZ-9HwQVTHjotzESVcKRpNJatSeYG_iwSH9LoT3F4to1nsMJVhAo0KgiSCDcu-zakpYE9XKxQeMeNfXF3Bdl5x-27GQrQGOWxYeek2q8tD', 'width': 3024}], 'place_id': 'ChIJw-sM1df0U0MRNC4G0I93Z8g', 'plus_code': {'compound_code': 'RX2C+GF Turukhansk, Krasnoyarsk Krai, Russia', 'global_code': '9MQ9RX2C+GF'}, 'rating': 3.7, 'reference': 'ChIJ

In [16]:
hotel 

['ООО "Туруханская пушнина"',
 'Abalone Lodge',
 'Top of the World Hotel',
 'The Prestige Hotel Penang',
 'Pingtian Peninsula Hotel',
 'Desert Gardens Hotel - Ayers Rock Resort',
 'Island Way Villa',
 'Frobisher Inn',
 'Höfði Cottages',
 nan,
 nan,
 'Pechora Gostinitsa',
 'NH Gran Hotel Provincial',
 'Southland Organic Farmstay',
 'Paradise First Hotel',
 'Blue Lagoon Hotel',
 'Los Cauquenes Resort & Spa',
 'Huayu Hotel',
 nan,
 'Hilton Port Moresby',
 nan,
 nan,
 nan,
 'The Explorer Hotel',
 'Abuharee Grand',
 'Hotel Castelo',
 nan,
 nan,
 nan,
 nan,
 nan,
 'Les Embruns Du Baril',
 'סטאי תל אביב - The Setai, Tel Aviv',
 'San Ramon Marriott',
 nan,
 'The Tasman, a Luxury Collection Hotel, Hobart',
 'De Hoop Collection',
 'Hotel Hacienda de Abajo',
 'Welk Resorts Cabo San Lucas - Sirena del Mar',
 'Hôtel Hiva Oa Hanakee Lodge',
 'Carnarvon Sea Change Apartments',
 nan,
 nan,
 'InterContinental Bora Bora Resort Thalasso Spa',
 'Descanso al Paso Chuy',
 'Hostal Plaza Lebu',
 nan,
 'Hotel 

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))